In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

#### Question 10

In [2]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Bookings LEFT JOIN Facilities USING(facid)")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

df.head()

,bookid,facid,memid,starttime,slots,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,3,1,2012-07-03 11:00:00,2,Table Tennis,0.0,5.0,320,10
1,1,4,1,2012-07-03 08:00:00,2,Massage Room 1,9.9,80.0,4000,3000
2,2,6,0,2012-07-03 18:00:00,2,Squash Court,3.5,17.5,5000,80
3,3,7,1,2012-07-03 19:00:00,2,Snooker Table,0.0,5.0,450,15
4,4,8,1,2012-07-03 10:00:00,1,Pool Table,0.0,5.0,400,15


In [3]:
def total_revenue(df):
    '''Calculates the revenue produced by each facility'''
    
    facilities = df['name'].unique()
    my_dict = {name:0 for name in facilities}
    
    for index, memid in enumerate(df['memid']):
        if memid == 0:
            my_dict[df['name'][index]] += df['slots'][index]*df['guestcost'][index]
        else:
            my_dict[df['name'][index]] += df['slots'][index]*df['membercost'][index]
    
    my_dict = {key: round(value, 2) for key, value in my_dict.items()}
    
    return my_dict

revenues = total_revenue(df)

print([key for key, value in revenues.items() if value < 1000])

['Table Tennis', 'Snooker Table', 'Pool Table']


#### Question 11

In [4]:
with engine.connect() as con:
    rs = con.execute("SELECT (m1.surname || ', ' || m1.firstname) AS member_name, "
                     "(m2.surname || ', ' || m2.firstname) AS recommender "
                     "FROM Members AS m1 LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
df.head()

,member_name,recommender
0,"GUEST, GUEST",None
1,"Smith, Darren",None
2,"Smith, Tracy",None
3,"Rownam, Tim",None
4,"Joplette, Janice","Smith, Darren"


In [5]:
report = df[df['recommender'].notna()]

report = report.sort_values('member_name')

report = report.reset_index()

report.drop('index',axis=1,inplace=True)

print(report)

                 member_name         recommender
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

#### Question 12

In [6]:
with engine.connect() as con:
    rs = con.execute("SELECT name, memid FROM Bookings "
                     "LEFT JOIN Facilities USING(facid)")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
df.head()

,name,memid
0,Table Tennis,1
1,Massage Room 1,1
2,Squash Court,0
3,Snooker Table,1
4,Pool Table,1


In [7]:
usage_report = df.groupby('name').agg({'memid':lambda x: sum(x!=0)})

usage_report.columns = ['Times Used By Members']

print(usage_report)

                 Times Used By Members
name                                  
Badminton Court                    344
Massage Room 1                     421
Massage Room 2                      27
Pool Table                         783
Snooker Table                      421
Squash Court                       195
Table Tennis                       385
Tennis Court 1                     308
Tennis Court 2                     276


#### Question 14

In [8]:
with engine.connect() as con:
    rs = con.execute("SELECT name, memid, starttime FROM Bookings "
                     "LEFT JOIN Facilities USING(facid)")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
df.head()

,name,memid,starttime
0,Table Tennis,1,2012-07-03 11:00:00
1,Massage Room 1,1,2012-07-03 08:00:00
2,Squash Court,0,2012-07-03 18:00:00
3,Snooker Table,1,2012-07-03 19:00:00
4,Pool Table,1,2012-07-03 10:00:00


In [9]:
df.columns = ['name','memid','month_year']

df['month_year'] = pd.to_datetime(df['month_year'])

df['month_year'] = df['month_year'].dt.strftime('%b-%Y')

usage_report_month = df.groupby(['name','month_year']).agg({'memid':lambda x: sum(x!=0)})

usage_report_month.columns =  ['Times Used By Members Per Month']

print(usage_report_month)

                            Times Used By Members Per Month
name            month_year                                 
Badminton Court Aug-2012                                132
                Jul-2012                                 51
                Sep-2012                                161
Massage Room 1  Aug-2012                                153
                Jul-2012                                 77
                Sep-2012                                191
Massage Room 2  Aug-2012                                  9
                Jul-2012                                  4
                Sep-2012                                 14
Pool Table      Aug-2012                                272
                Jul-2012                                103
                Sep-2012                                408
Snooker Table   Aug-2012                                154
                Jul-2012                                 68
                Sep-2012                